In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.1/887.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 934.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00


#데이터 로드 및 전처리

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sampled_5000_matches.csv')

In [ ]:
df.head()

,date,game_size,match_id,match_mode,party_size,player_assists,player_dbno,player_dist_ride,player_dist_walk,player_dmg,player_kills,player_name,player_survive_time,team_id,team_placement
0,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,0,0.0,2115.047610,19,0,ssdaybreak,1084.986,1,12
1,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,0,0.0,2663.025630,0,0,Hung0,1185.968,1,12
2,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,1,0.0,75.278170,138,1,Red_Acher,158.299,6,26
3,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,0,0.0,26.331602,0,0,2pacTupac,130.263,6,26
4,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,1,0.0,77.084120,175,1,qweaaaa2,157.039,6,26


##GNN 기반

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GAE
from tqdm import tqdm

# ------------------------------------------
# 1. 데이터 전처리: 플레이어별 평균 스탯 집계
# ------------------------------------------
# 예시: df DataFrame에 다음 컬럼들이 있다고 가정합니다.
# date, game_size, match_id, match_mode, party_size,
# player_assists, player_dbno, player_dist_ride, player_dist_walk,
# player_dmg, player_kills, player_name, player_survive_time, team_id, team_placement

# 플레이어별 평균 스탯 집계 (여기서는 간단하게 평균값 사용)
agg_dict = {
    "player_kills": "mean",
    "player_assists": "mean",
    "player_dbno": "mean",
    "player_dist_ride": "mean",
    "player_dist_walk": "mean",
    "player_survive_time": "mean"
}
df_agg = df.groupby("player_name").agg(agg_dict).reset_index()

# 총 이동 거리 계산
df_agg["total_distance"] = df_agg["player_dist_ride"] + df_agg["player_dist_walk"]

# 사용할 피처 선택
feature_cols = ["player_kills", "player_assists", "player_dbno", "total_distance", "player_survive_time"]
x_np = df_agg[feature_cols].values.astype(np.float32)
x = torch.tensor(x_np, dtype=torch.float)

# ------------------------------------------------------
# 2. 그래프 구성: 노드는 플레이어, 엣지는 같은 팀원 간 연결
# ------------------------------------------------------
# 각 매치(match_id)와 팀(team_id)별로 플레이어 이름을 그룹화하여, 같은 팀원 간에 엣지를 생성합니다.
# df에서 매치(match_id)와 팀(team_id)별로 플레이어 이름을 그룹화하여, 같은 팀원 간에 엣지를 생성합니다.
edges = set()
for (match_id, team_id), group in df.groupby(["match_id", "team_id"]):
    # player_name 컬럼을 문자열로 변환한 후 고유값 추출 (NaN 제거)
    players = group["player_name"].dropna().astype(str).unique()
    # 팀 내 모든 플레이어 쌍에 대해 무방향 엣지 생성 (완전 연결 그래프)
    for i in range(len(players)):
        for j in range(i+1, len(players)):
            # 두 플레이어 이름을 문자열 비교가 가능하도록 정렬
            a, b = sorted([players[i], players[j]])
            edges.add((a, b))

# 플레이어 이름을 인덱스로 매핑 (노드 순서 결정)
name_to_idx = {name: idx for idx, name in enumerate(df_agg["player_name"].values)}

edge_index_list = []
for a, b in edges:
    # 두 플레이어 모두 집계 데이터에 존재할 경우에만 추가
    if a in name_to_idx and b in name_to_idx:
        edge_index_list.append([name_to_idx[a], name_to_idx[b]])
        edge_index_list.append([name_to_idx[b], name_to_idx[a]])  # 무방향이므로 양쪽 모두

edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()  # shape: [2, num_edges]

# PyTorch Geometric Data 객체 생성
data = Data(x=x, edge_index=edge_index)

# ------------------------------------------------------
# 3. GNN 모델 정의: GCN 기반 Encoder와 GAE 활용
# ------------------------------------------------------
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

embedding_dim = 1  # 1차원 임베딩 → 복합 성과 점수로 해석 가능
encoder = GCNEncoder(in_channels=x.shape[1], out_channels=embedding_dim)
model = GAE(encoder)

optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 100

# ------------------------------------------------------
# 4. 학습: Graph Autoencoder를 이용해 노드 임베딩 학습 (tqdm 진행바 포함)
# ------------------------------------------------------
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    # 전체 그래프를 입력으로 임베딩 추출
    z = model.encode(data.x, data.edge_index)
    loss = model.recon_loss(z, data.edge_index)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# ------------------------------------------------------
# 5. 임베딩(복합 성과 점수) 추출 및 DataFrame에 병합
# ------------------------------------------------------
model.eval()
with torch.no_grad():
    z = model.encode(data.x, data.edge_index)
# embedding_dim이 1이므로 squeeze하여 1차원 배열로 만듭니다.
composite_scores = z.squeeze().numpy()

# df_agg에 GNN으로 학습한 복합 성과 점수 추가
df_agg["composite_performance_score_gnn"] = composite_scores

Epoch 10/100, Loss: 34.5388
Epoch 20/100, Loss: 34.5388
Epoch 30/100, Loss: 34.5388
Epoch 40/100, Loss: 34.5388
Epoch 50/100, Loss: 34.5388
Epoch 60/100, Loss: 34.5388
Epoch 70/100, Loss: 34.5388
Epoch 80/100, Loss: 34.5388
Epoch 90/100, Loss: 34.5388
Epoch 100/100, Loss: 34.5388


In [ ]:
df.head()

,date,game_size,match_id,match_mode,party_size,player_assists,player_dbno,player_dist_ride,player_dist_walk,player_dmg,player_kills,player_name,player_survive_time,team_id,team_placement,total_distance,composite_performance_score
0,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,0,0.0,2115.047610,19,0,ssdaybreak,1084.986,1,12,2115.047610,-2.514812
1,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,0,0.0,2663.025630,0,0,Hung0,1185.968,1,12,2663.025630,-2.458499
2,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,1,0.0,75.278170,138,1,Red_Acher,158.299,6,26,75.278170,-3.027798
3,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,0,0.0,26.331602,0,0,2pacTupac,130.263,6,26,26.331602,-23.213470
4,2018-01-03T23:23:03+0000,27,2U4GBNA0YmlmN9OV04IhlvWAEi5_umKbgD3OYbUyA1eaSh...,tpp,4,0,1,0.0,77.084120,175,1,qweaaaa2,157.039,6,26,77.084120,-3.028286


In [ ]:
len(df['player_name'].unique())

393286

##라벨링 작업

In [ ]:
import pandas as pd
import numpy as np

# 예시: 이미 df라는 DataFrame이 있다고 가정
# df의 컬럼: date, game_size, match_id, match_mode, party_size,
# player_assists, player_dbno, player_dist_ride, player_dist_walk,
# player_dmg, player_kills, player_name, player_survive_time, team_id, team_placement

'''
# =============================================================================
# 1. 각 매치에서의 성과 스코어 계산 (여기서는 간단히 player_kills를 성과로 사용)
# =============================================================================
# 1. 총 이동 거리 계산 (라이드 + 워크)
df["total_distance"] = df["player_dist_ride"] + df["player_dist_walk"]

# 2. 각 요소별 가중치 설정 (이 값들은 도메인에 맞게 튜닝 필요)
weight_kills        = 100   # 킬에 높은 가중치 부여
weight_dbno         = 50    # DBNO(Down But Not Out) 횟수
weight_assists      = 25    # 어시스트
weight_distance     = 0.05  # 총 이동 거리 (단위: 미터)
weight_survive_time = 0.1   # 생존 시간 (단위: 초)

# 3. 복합 성과 지표 performance_score 계산
df["performance_score"] = (
    weight_kills * df["player_kills"] +
    weight_dbno * df["player_dbno"] +
    weight_assists * df["player_assists"] +
    weight_distance * df["total_distance"] +
    weight_survive_time * df["player_survive_time"]
)
'''

# =============================================================================
# 2. 플레이어별 평소 실력(기본 성과) 산출: 전체 매치에서의 평균 성과 (예: 평균 킬 수)
# =============================================================================
player_baseline = (
    df.groupby("player_name")["composite_performance_score"]
      .mean()
      .reset_index()
      .rename(columns={"composite_performance_score": "baseline_score"})
)

# =============================================================================
# 3. 각 매치 행(row)에 플레이어의 평소 실력을 합치기 위해 baseline과 병합
# =============================================================================
df = df.merge(player_baseline, on="player_name", how="left")

# =============================================================================
# 4. 각 매치에서의 성과 편차 계산
#    편차 = 해당 매치 성과 - 플레이어 평소 평균 성과
# =============================================================================
df["performance_deviation"] = df["composite_performance_score"] - df["baseline_score"]

# 또한 상대적 성과 비율 계산 (평소 대비 몇 배의 성과를 내었는지)
df["performance_ratio"] = df["composite_performance_score"] / (df["baseline_score"] + 1e-6)  # 0으로 나누는 오류 방지

# =============================================================================
# 5. 같은 매치(match_id) 내 같은 팀(team_id)끼리 팀원들의 성과를 이용해 팀 강약(팀원 평균 성과)을 계산
#    - 각 그룹 내에서, 자신을 제외한 나머지 플레이어의 평균 성과를 구합니다.
# =============================================================================
def compute_team_strength(series):
    total = series.sum()
    count = series.count()
    # 팀원이 1명인 경우는 계산할 수 없으므로 NaN 처리
    return series.apply(lambda x: (total - x) / (count - 1) if count > 1 else np.nan)

df["team_strength"] = df.groupby(["match_id", "team_id"])["composite_performance_score"].transform(compute_team_strength)

# =============================================================================
# 6. 약한 팀(weak team) 판별
#    - 전체 팀 강약 분포에서 하위 30% 이하를 약팀으로 설정 (이 값은 조정 가능)
# =============================================================================
weak_threshold = df["team_strength"].quantile(0.3)
df["weak_team"] = df["team_strength"] < weak_threshold

# =============================================================================
# 7. 약팀에서의 플레이어 성과 편차 및 상대적 성과 비율을 플레이어별로 집계
# =============================================================================
# 약팀 매치에서의 평균 성과 편차 (절대값)
player_weak_deviation = (
    df[df["weak_team"]]
    .groupby("player_name")["performance_deviation"]
    .mean()
    .reset_index()
)

# 약팀 매치에서의 평균 성과 비율
player_weak_ratio = (
    df[df["weak_team"]]
    .groupby("player_name")["performance_ratio"]
    .mean()
    .reset_index()
)

# =============================================================================
# 8. 플레이어 기본 성과와 약팀에서의 성과 지표를 병합하여 분석용 DataFrame 생성
# =============================================================================
player_analysis = (
    player_baseline
    .merge(player_weak_deviation, on="player_name", how="left")
    .merge(player_weak_ratio, on="player_name", how="left")
    .rename(columns={
        "baseline_score": "avg_score_overall",
        "performance_deviation": "avg_deviation_weak",
        "performance_ratio": "avg_ratio_weak"
    })
)

# =============================================================================
# 9. 평소에는 준수한 실력을 가지지만 약팀에서 급격히 성과가 떨어지는 유저 선별
#    - 예: 평소 평균 성과가 0보다 크고, 약팀에서의 성과 비율이 0.5 미만인 유저
# =============================================================================
potential_throwers = player_analysis[
    (player_analysis["avg_score_overall"] > 0) & (player_analysis["avg_ratio_weak"] < 0.5)
].sort_values("avg_ratio_weak")

# 결과 출력
print("잠재적 게임 던지는(트롤) 유저:")
print(potential_throwers)

잠재적 게임 던지는(트롤) 유저:
            player_name  avg_score_overall  avg_deviation_weak  avg_ratio_weak
379042  xuanyunmingfeng           0.000093           -2.178993   -23272.476562
202343     SegwayJesus_           0.003831           -2.942895     -767.029236
104942       HoldMePapi           0.003571           -2.425426     -678.049866
198937           SY_YHC           0.004813           -3.046978     -631.884338
5278         2904713218           0.008238           -3.026577     -366.360291
...                 ...                ...                 ...             ...
13781         APPLEPEET           1.121171           -0.798143        0.288116
379088            xucoo           9.073892           -5.789828        0.361924
145410            Lyenn           1.139013           -0.636236        0.441414
232883      UchihaObito           2.306042           -1.279980        0.444945
157815           Mlenys           1.466952           -0.764696        0.478717

[801 rows x 4 columns]


In [ ]:
len(potential_throwers['player_name'].unique())
# 1000명중에 2명꼴로 트롤

801